In [1]:
import sys, os, re

sys.path = (["../src/", "../"] if re.match(r"^(\w\:\\)|(/)", os.getcwd()) else []) + sys.path
import qubx
from qubx import logger

%qubxd

%load_ext autoreload
%autoreload 2

from qubx.core.basics import Signal, TimestampedDict
from qubx.core.interfaces import IStrategy, IStrategyContext, TriggerEvent, MarketEvent
from qubx.data.readers import MultiTypeReader
from qubx.pandaz.utils import *

from qubx.backtester.simulator import simulate


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.1
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


## <font color='#00ff00'>Multi type data</font>

In [5]:
class TestB(IStrategy):
    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent) -> list[Signal] | Signal | None:
        logger.info(f'{data.instrument} market event ::: <g>{data.type}</g> ::: -> {data.data}')

In [6]:
idx = pd.date_range(start="2023-06-01 00:00", end="2023-06-01 01:00", freq="1s", name="timestamp")

qts_raw = [
    TimestampedDict(t, {"bid": i, "ask": i, "bid_vol": 100, "ask_vol": 200}) for i, t in enumerate(idx)
]
trades_raw = [
    TimestampedDict(t, {"price": i, "size": 100}) for i, t in enumerate(idx)
]
ob_raw = [
    TimestampedDict(t, {
        "asks": [(100 + i/100, 100) for i in range(100)], 
        "bids": [(100 - i/100, 100) for i in range(100)],
    })
    for t in idx
]
custom_raw = [
    TimestampedDict(t, {"what_to_do": np.random.choice(["buy", "sell"]), "how_much": i})
    for i, t in enumerate(idx)
]

reader = MultiTypeReader(
    {
        "BINANCE.UM:BTCUSDT": {
            "quote": qts_raw,
            "trade": trades_raw,
            "orderbook": ob_raw,
            "CUSTOM": custom_raw,
        }
    }
)

In [7]:
r = simulate({'Test 1': TestB(show_mkt=1)}, 
   {  
      'quote': reader, 
      'trade': reader, 
      'orderbook': reader, 
      'CUSTOM': reader, 
    },

   1000, ['BINANCE.UM:BTCUSDT'], "vip0_usdt", "2023-06-01 00:00", "2023-06-01 00:01", debug="DEBUG"
)

2023-06-01 00:00:00.000 [🐞] [simulator] :: Preparing simulated trading on BINANCE.UM for 1000 USDT...
2023-06-01 00:00:00.000 [ℹ️] SimulatedDataProvider.binance.um is initialized


Simulating:   0%|          | 0/100 [00:00<?, ?%/s]

2023-06-01 00:00:00.000 [🐞] [simulator] :: Setting up default base subscription: quote
2023-06-01 00:00:00.000 [🐞] [simulator] :: Strategy didn't set warmup period for quote so default 1Min will be used
2023-06-01 00:00:00.000 [🐞] [simulator] :: Subscribing to: quote
2023-06-01 00:00:00.000 [🐞] [simulator] :: Subscribing to: trade
2023-06-01 00:00:00.000 [🐞] [simulator] :: Subscribing to: orderbook
2023-06-01 00:00:00.000 [🐞] [simulator] :: Subscribing to: CUSTOM
2023-06-01 00:00:00.000 [ℹ️] SimulatedDataProvider ::: Simulation started at 2023-06-01 00:00:00 :::
2023-06-01 00:00:00.000 [🐞]   [IteratedDataStreamsSlicer] :: Preloading initial data for quote 2023-06-01 00:00:00 : 2023-06-01 00:01:00 ...
2023-06-01 00:00:00.000 [🐞] [ProcessingManager] :: Invoking TestB on_fit
2023-06-01 00:00:00.000 [🐞] [ProcessingManager] :: TestB is fitted
2023-06-01 00:00:00.000 [ℹ️] BINANCE.UM:FUTURE:BTCUSDT market event ::: CUSTOM ::: -> TimestampedDict(time=1685577600000000000, data={'timestamp': Tim